In [2]:
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.loaders import UnstructuredIO
from camel.storages import Neo4jGraph
from camel.retrievers import AutoRetriever
from camel.embeddings import MistralEmbedding
from camel.types import StorageType
from camel.agents import ChatAgent, KnowledgeGraphAgent
from camel.messages import BaseMessage

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()

DEEPSEEK_API_KEY = os.getenv("DEEPSEEK_API_KEY")
DEEPSEEK_API_BASE = "https://api.deepseek.com"

In [4]:
# Configuring Neo4j Graph Database
n4j = Neo4jGraph(
    url="bolt://10.140.52.87:7687",
    username="neo4j",
    password="123456789",
)

In [5]:
# creating the model
deepseek = ModelFactory.create(
    model_platform=ModelPlatformType.DEEPSEEK,
    api_key=DEEPSEEK_API_KEY,
    url=DEEPSEEK_API_BASE,
    model_type=ModelType.DEEPSEEK_CHAT,
    model_config_dict={"max_tokens": 4096},
)

In [6]:
# Generate a Knowledge Graph Using CAMEL's Agent

# Set instance
uio = UnstructuredIO()
kg_agent = KnowledgeGraphAgent(model=deepseek)

In [7]:
# Set example text input
text_example = """
CAMEL has developed a knowledge graph agent can run with Mistral AI's most
advanced model, the Mistral Large 2. This knowledge graph agent is capable
of extracting entities and relationships from given content and create knowledge
graphs automaticlly.
"""

In [8]:
# Create an element from given text
element_example = uio.create_element_from_text(text=text_example, element_id="0")

In [9]:
# Let Knowledge Graph Agent extract node and relationship information

ans_element = kg_agent.run(element_example, parse_graph_elements=False)
print(ans_element)

### Extracted Nodes and Relationships:

#### Nodes:
1. **Node(id='CAMEL', type='Organization')**  
   - Represents the organization that developed the knowledge graph agent.

2. **Node(id='Knowledge Graph Agent', type='Software')**  
   - Represents the software developed by CAMEL.

3. **Node(id='Mistral AI', type='Organization')**  
   - Represents the organization that developed the Mistral Large 2 model.

4. **Node(id='Mistral Large 2', type='Model')**  
   - Represents the advanced model developed by Mistral AI.

#### Relationships:
1. **Relationship(subj=Node(id='CAMEL', type='Organization'), obj=Node(id='Knowledge Graph Agent', type='Software'), type='Developed')**  
   - Represents the relationship where CAMEL developed the Knowledge Graph Agent.

2. **Relationship(subj=Node(id='Knowledge Graph Agent', type='Software'), obj=Node(id='Mistral Large 2', type='Model'), type='RunsWith')**  
   - Represents the relationship where the Knowledge Graph Agent runs with the Mistral Large 2

In [10]:
# Check graph element
graph_elements = kg_agent.run(element_example, parse_graph_elements=True)
print(graph_elements)

nodes=[Node(id='CAMEL', type='Organization', properties={'source': 'agent_created'}), Node(id='Knowledge Graph Agent', type='Software', properties={'source': 'agent_created'}), Node(id='Mistral AI', type='Organization', properties={'source': 'agent_created'}), Node(id='Mistral Large 2', type='Model', properties={'source': 'agent_created'}), Node(id='Entities', type='Concept', properties={'source': 'agent_created'}), Node(id='Relationships', type='Concept', properties={'source': 'agent_created'}), Node(id='Knowledge Graphs', type='Concept', properties={'source': 'agent_created'})] relationships=[Relationship(subj=Node(id='CAMEL', type='Organization', properties={'source': 'agent_created'}), obj=Node(id='Knowledge Graph Agent', type='Software', properties={'source': 'agent_created'}), type='Developed', properties={'source': 'agent_created'}), Relationship(subj=Node(id='Knowledge Graph Agent', type='Software', properties={'source': 'agent_created'}), obj=Node(id='Mistral Large 2', type='M

In [11]:
# Add the element to neo4j database
n4j.add_graph_elements(graph_elements=[graph_elements])